In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm

import xarray as xr


## Data Loading

In [56]:
def process_it(ds, date):
    ds1 = ds.assign(time=("TSTEP", pd.date_range(date, periods=24, freq="H")))
    lats = np.arange(80) * ds1.YCELL + ds1.YORIG
    lons = np.arange(80) * ds1.XCELL + ds1.XORIG
    ds2 = ds1.assign_coords(lat=("ROW", lats), lon=("COL", lons))
    ds3 = ds2.swap_dims({"TSTEP": "time", "ROW": "lat", "COL": "lon"})
    return ds3


drange = pd.date_range(start="2023-07-17", end="2023-08-16", freq="D").strftime(
    "%Y%m%d"
)
aq_data = []
for date in tqdm(drange):
    ds = xr.open_dataset(f"../../sarath_auto_download/data/camx120hr_merged_{date}.nc")
    met_ds = xr.open_dataset(
        f"../../sarath_auto_download/data/camxmet2d.delhi.{date}.96hours.nc"
    )
    if ds.TSTEP.size == 120:
        ds = ds.isel(LAY=0, TSTEP=range(24, 48))[["P25"]]
    else:
        ds = ds.isel(LAY=0, TSTEP=range(24))[["P25"]]

    assert met_ds.TSTEP.size == 96
    met_ds = met_ds.isel(TSTEP=range(24))

    ds = process_it(ds, date)

    aq_data.append(ds)

aq_data = xr.concat(aq_data, dim="time")
aq_data

100%|██████████| 31/31 [00:00<00:00, 255.70it/s]


<xarray.Dataset>
Dimensions:  (time: 744, lat: 80, lon: 80)
Coordinates:
  * time     (time) datetime64[ns] 2023-07-17 ... 2023-08-16T23:00:00
  * lat      (lat) float64 28.2 28.21 28.22 28.23 ... 28.96 28.97 28.98 28.99
  * lon      (lon) float64 76.85 76.86 76.87 76.88 ... 77.61 77.62 77.63 77.64
Data variables:
    P25      (time, lat, lon) float32 23.63 23.63 25.39 ... 32.28 31.11 31.11
Attributes: (12/34)
    IOAPI_VERSION:  $Id: @(#) ioapi library version 3.0 $                    ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2023197
    CTIME:          83911
    WDATE:          2023197
    ...             ...
    GDNAM:          ????????????????
    UPNAM:          CAMXMETOU       
    VAR-LIST:       P10             P25             
    FILEDESC:       I/O API formatted CAMx AVRG output                       ...
    HISTORY:        Mon Jul 17 08:45:22 2023: ncrcat camxout.2023.07.16.nc ca...
    NCO:            netCDF Operators version 4.9.1 (Homepage = http://nco.sf....

In [59]:
xr.open_dataset("../../active_learn/caaqm_data/delhi_cpcb_2023.nc").time


<xarray.DataArray 'time' (time: 5064)>
array(['2023-01-01T00:30:00.000000000', '2023-01-01T01:30:00.000000000',
       '2023-01-01T02:30:00.000000000', ..., '2023-07-30T21:30:00.000000000',
       '2023-07-30T22:30:00.000000000', '2023-07-30T23:30:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2023-01-01T00:30:00 ... 2023-07-30T23:30:00